# 7.1 LangChain基础模块

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_BASE"]  = "xxx"
os.environ["OPENAI_API_KEY"] = "xxx"
chat = ChatOpenAI()
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
    HumanMessage(content="I love programming."),
]
chat(messages)

AIMessage(content='我喜欢编程。')

In [1]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./test.txt")
print(loader.load())

[Document(page_content='Today is Sunday.\nI am a boy.\n', metadata={'source': './test.txt'})]


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
import os
os.environ["OPENAI_API_BASE"]  = "xxx"
os.environ["OPENAI_API_KEY"] = "xxx"

raw_documents = [Document(page_content="葡萄", metadata={"source": "local"}),
Document(page_content="白菜", metadata={"source": "local"}),
Document(page_content="狗", metadata={"source": "local"})]
db = Chroma.from_documents(raw_documents, OpenAIEmbeddings())
query = "动物"
docs = db.similarity_search(query)
print(docs[0].page_content)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


狗


# 7.2.基于LangChain实现RAG

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
import os

os.environ["OPENAI_API_BASE"]  = "xxx"
os.environ["OPENAI_API_KEY"] = "xxx"

loader = TextLoader('7.2_RAG_demo.txt',encoding = "utf-8")
documents = loader.load()
text_splitter = \
RecursiveCharacterTextSplitter(
    chunk_size=10, chunk_overlap=0,separators=["\n"])
chunks = text_splitter.split_documents(documents)
template = """你是一位问答助手,你的任务是根据###中间的文本信息回答问题，请准确回答问题，不要健谈，如果提供的文本信息无法回答问题，请直接回复“提供的文本无法回答问题”，我相信你能做的很好。###\n{context}###\n问题：{question}"""
question = "战士金喜欢哪写乐队？"
db = Chroma.from_documents(chunks, OpenAIEmbeddings())
retriever = db.as_retriever(search_kwargs={"k": 2})
context =  retriever.get_relevant_documents(question)
print(context)

D:\Anaconda\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[Document(page_content='\n谷清水的昵称是战士金', metadata={'source': '7.2_RAG_demo.txt'}), Document(page_content='\n谷清水是万能青年旅店、腰、华北浪革、生祥、声音碎片等乐队的粉丝', metadata={'source': '7.2_RAG_demo.txt'})]


In [6]:
context_str = "；".join([d.page_content for d in context])
input_str = template.format_map({"context":context_str,"question":question})
chat = ChatOpenAI()
messages = [SystemMessage(content="你是一位问答助手"),HumanMessage(content=input_str)]
response = chat(messages)
print(response)

D:\Anaconda\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='战士金喜欢万能青年旅店、腰、华北浪革、生祥、声音碎片等乐队。'
